NEW
* matchlist+names union (only names with 3 or more chars) + 'ja' which is part of words that cannot start the pw
* get all correct matches; treshold based on Levenshtein distance and other measures
* best matches, name/word, morpholocigal analysis

**TKO_2023 Exercise Project 5 cp**<br>
**Password analysis**<br>
Halonen Marko, Syrjälä Elise

\newpage \tableofcontents \newpage

***TKO_2023 Exercise Project 5 cp***

**Password analysis**<br>
Halonen Marko, Syrjälä Elise

## Short description

This Python notebook contains mains steps for data preparation and feature generation.
Aim is to analyze what kind of passwords are used in Finnish network sites. The leaked passwords have been retrieved from trusted source and data has been treted with confinedtial handling; restricted utu Linux server, and when needed transferred to seafile.utu.fi server, synced also to local copy, only to password protected computers with encrypted hardrives.

## Planned steps

1. read pw file, including ready matched words (matching can be tuned later, if needed)
2. get list of matched words that are Finnish; intersection of matched words and omor_uniq
3. get list of matched word overal frequencies (and pos + omor analysis); intersection of matched words and all_vocab, inlc frequency, pos and analysis
4. get list of best matches using algorith that finds the longest matching combinations, incl double words
5. extract to datasets
6. analyze

## Source files

PW (matched lists *.txt.matched)

* source1
* source2
* source3
* source4
* source5
* source6
* source7
* source8
* source9

List if Finnish words

* filtered_vocab_omor.gz (two columns, word in first, analysis in latter, lots of empty rows)
* filtered_vocab_omor_uniq-gz (only one column of uniq Finnish words)

Word Frequency

* all_vocab (web crawled data that has word, pos, morphological analysis and freq) xxxx this source needs fixing

## Source of the passwords

In [1]:
'''based on these given names
1. source file is extracted
2. datafile is stored in the end of the pipeline'''
pwfilename='matched/78k-all-passwords.txt.matched'
sourcename='78k'

# Reading the passwords and preliminary matches

## Importing the data

### Import to dictionary format

Import matched data (pw + matches) to dictionary format. PW as the key. 

Name: 
* d0: password as key, all possible matches as items

In [2]:
import gzip
import json

Some files gzipped, some are not, two options therefore provided

In [3]:
#file=open('')
def gz_pwmatches_todict(filename):
    '''reads zipped file that has pw in first column,
    and list of matches in next column,
    returns dictionary where the pw is key.
    Note that also pws with no math are included.'''
    dictx={}
    with gzip.open(filename, 'rb') as f:
        for line in f:
            pw, lst=line.split("\t")
            match=json.loads(lst)        
            dictx[pw]=match
    f.close()
    return dictx

In [4]:
#file=open('')
def pwmatches_todict(filename):
    '''reads file that has pw in first column,
    and list of matches in next column,
    returns dictionary where the pw is key.
    Note that also pws with no math are included.'''
    dictx={}
    with open(filename, 'rb') as f:
        for line in f:
            pw, lst=line.split("\t")
            match=json.loads(lst)
            dictx[pw]=match
    f.close()
    return dictx

In [5]:
# wordlist=[x.decode('utf-8') for x in wordtemp] # already in unicode, this is not needed

In [6]:
'''Running the procedure with source given'''
d0=pwmatches_todict(pwfilename)

In [7]:
'''test print, getting 
a) the last key value 
b) last key related item values and 
c) first key and item values together'''
d0.keys()[-1],d0[d0.keys()[-1]], d0.items()[0]

('Tammikuu0505', [u'tammikuu', u'kuu', u'tammi'], ('syrkky69', []))

In [8]:
# length/size of dictionary, number of items in first key
len(d0), len(d0.values()[0])

(43870, 0)

In [9]:
# '''check'''
# [k for k in d0.iterkeys() if '@' in k], [k for k in d0.iterkeys() if ',' in k]

In [10]:
# '''Remove all keys and values that contain char @'''
# d0 = {k:v for k,v in d0.items() if '@' not in k}

In [11]:
len(d0), len(d0.values()[0])

(43870, 0)

In [12]:
'''Checklist is a list that has more than one item per key.
This kind of list needs to be treated a bit differently in later processing.
Printing out also a list for visual inspection'''
checklist=[item for item in d0.values() if len(item)>1]
checklist[:20]

[[u'uni', u'kaunis', u'kaunisto'],
 [u'jussi', u'jussila', u'silat'],
 [u'kulta', u'tara', u'rakas'],
 [u'petra', u'petr'],
 [u'protacon', u'ota', u'pro', u'prota'],
 [u'new', u'cool', u'news'],
 [u'ruuna', u'marjut', u'sitruuna', u'marju', u'sit'],
 [u'tela', u'este'],
 [u'mill', u'camilla', u'milla', u'viivi'],
 [u'nalle', u'all', u'alle'],
 [u'aave', u'taavetti'],
 [u'herne', u'herkku'],
 [u'nalle', u'all', u'alle'],
 [u'kawasaki', u'aki', u'was'],
 [u'mba', u'sam', u'samu'],
 [u'oli', u'tipsu', u'viat', u'olivia', u'sukka', u'livia'],
 [u'ura', u'burana', u'uran'],
 [u'mia', u'aija', u'maija'],
 [u'sam', u'sama'],
 [u'ulla', u'pulla']]

In [13]:
'''Checklist2 is a list that has only one item per key.
This kind of list needs to be treated a bit differently in later processing.
Printing out a short list for visual inspection'''
checklist2=[item for item in d0.values() if len(item)==1]
checklist2[:5]

[[u'sonja'], [u'enne'], [u'koira'], [u'elijah'], [u'timo']]

In [14]:
'''Checking
a) key and values of the second observation
b) checklist for the same value (both as not sure which list for different pw source files)
c) type of the values, which should be unicode'''
print d0.items()[0], d0.values()[0]
print checklist[0][0], checklist2[0][0]
print type(checklist[0][0]), type(checklist2[0][0])

('syrkky69', []) []
uni sonja
<type 'unicode'> <type 'unicode'>


### Import to list format

Importing passwords and all found matches to lists that have corresponding indexing.

Files:
* pwlist - password
* matchlist - list of all possible mathces

List format import included, as the eventual analysis part was planned for lists and does not work for dictionaries as such.

If 'Type:' is found, the occurrence is not included in the analysis.
Example:
"[u'Type: text/html Subject: on, good, as mayeas, that and bcc: homestead7313@werndgains.com aphod, hat s very fast. he jumping quite the eart of had always help him so redly on missiles do you may it and ei']"

In [15]:
def gz_pwmatches_tolists(filename):
    '''reads gz file that has pw in first column,
    and list of matches in next column,
    returns two lists: list of passwords and list of matches.
    Note that also pws with no math are included.'''
    allpws=[]
    pwtemplist=[]
    matchlist = []
    with gzip.open(filename, 'rb') as f:
        for line in f:
            word, lst =line.split("\t")
            lst=json.loads(lst)
#             if word.find('6') != -1:
#                 print word
            if lst==[] or word.find('Type:') != -1 or word.find('&#9679') != -1 or \
            word.find('&#1096') != -1 or word.find('&#9679') != -1 or word.find('&#9688') != -1:
                continue
            else:          
                allpws.append((word))
                pwtemplist.append((word))
                matchlist.append((lst))
    f.close()

    # wordlist in unicode
    pwlist=[x.decode('utf-8') for x in pwtemplist] 
    
    return pwlist, matchlist

In [16]:
def pwmatches_tolists(filename):
    '''reads file that has pw in first column,
    and list of matches in next column,
    returns two lists: list of passwords and list of matches.
    Note that also pws with no math are included.'''
    allpws=[]
    pwtemplist=[]
    matchlist = []
    with open(filename, 'rb') as f:
        for line in f:
            word, lst =line.split("\t")
            lst=json.loads(lst)
#             if word.find('6') != -1:
#                 print word
            if lst==[] or word.find('Type:') != -1 or word.find('&#9679') != -1 or \
            word.find('&#1096') != -1 or word.find('&#9679') != -1 or word.find('&#9688') != -1 or \
            word.find('&#9618') != -1:
                continue
            else:          
                allpws.append((word))
                pwtemplist.append((word))
                matchlist.append((lst))
    f.close()

    # wordlist in unicode
    allpws=[x.decode('utf-8') for x in allpws]
    pwlist=[x.decode('utf-8') for x in pwtemplist]
    
    return pwlist, matchlist, allpws

In [17]:
pwlist, matchlist, allpws = pwmatches_tolists(pwfilename)

In [18]:
'''checking that the length, should be identical numbers'''
print len(pwlist), len(matchlist), len(allpws)

38395 38395 38395


In [19]:
from collections import Counter

'''count all occurrences of unique pws'''

allpw_counter = Counter(allpws)
Counter(allpws).most_common(10)

[(u'salasana', 169),
 (u'perkele', 112),
 (u'qwerty', 64),
 (u'rakkaus', 56),
 (u'enkeli', 45),
 (u'paska', 37),
 (u'killer', 36),
 (u'kakka', 32),
 (u'joonas', 31),
 (u'password', 31)]

In [20]:
[k for k in pwlist if 'Type:' in k]

[]

In [21]:
[k for k in allpws if '@' in k],[k for k in allpws if '\"' in k]

([u'@@@@vector',
  u'@kata@katja11',
  u'@lovex@moi123',
  u'@jossu@inkku1',
  u'@rm1hevonen',
  u'with1891@battle.fi',
  u'n@@ranoora1995',
  u'nastjanastja@',
  u'noor@moimoi',
  u'^misu@^sika',
  u'esignation1990@battle.fi',
  u'from7450@kellotuspaja.comfrom7450@kellotuspaja.com',
  u'jones@@joonas12',
  u'melu@koira',
  u'mir@janna',
  u'n@llefrendit66',
  u'lucy199@',
  u'reki_pukki@hotmail.com123456',
  u'sergei!@11sergei11',
  u'sergei@11sergei11',
  u'temperature8141@battle.fi',
  u'tiku.poika@tiku',
  u'veer@paska93',
  u'\xe9lisakehela@hotmil.opi123456',
  u'pepsigirl@145',
  u'1t1h3x8@Lucas',
  u'joo@hmm1'],
 [u'Telewell1\\"', u'turhaa!"#'])

In [22]:
[k for k in allpws if ';' in k]

[u'&#9829;perkele', u'jenna;,pumpuli']

In [23]:
len(pwlist), len(matchlist)

(38395, 38395)

In [24]:
'''checkprint; two first passwords'''
print pwlist[:2]

[u'f1leop', u'lapavesi67']


In [25]:
'''checkprint; two first matchlists, additionally the first value of the second list '''
print matchlist[:2], matchlist[1][0], pwlist[:2], pwlist[1], allpws[:2]

[[u'leo'], [u'lapa', u'vesi']] lapa [u'f1leop', u'lapavesi67'] lapavesi67 [u'f1leop', u'lapavesi67']


### YYY

# Identify the Finnish words

## Read the namefile

In [26]:
namelist=[]
with open('names.txt', 'rb') as f:
    for line in f:
        name=line.rstrip("\n,\r") # remove extra chars
        '''\n - ASCII Linefeed (LF)
        \r - ASCII Carriage Return (CR)'''
        namelist.append(name)

f.close()

namelist=[item.decode('unicode-escape') for item in namelist]
namelist=[item.lower() for item in namelist]
namelist=[item for item in namelist if len(item)>2] # use only if length is more than 2 chars

### Checks

In [27]:
print len(namelist), type(namelist[0]), namelist[0]

34024 <type 'unicode'> maria


In [28]:
print namelist[-10:], namelist[:10]

[u'ylivirta', u'yl\xe4-lahti', u'yl\xe4-rotiala', u'zeus', u'\xe5kermarck', u'\xe5kman', u'\xe4mt\xf6', u'\xe4n\xe4k\xe4inen', u'\xf6rth\xe9n', u'\xf6ykk\xf6nen'] [u'maria', u'helena', u'anneli', u'johanna', u'kaarina', u'marjatta', u'hannele', u'kristiina', u'liisa', u'elina']


In [29]:
print u'Tuulia'.lower() in namelist, u'Tuulia' in namelist

True False


In [30]:
nameset = set(namelist)

'''add ja to the nameset as that short words were not included in matchsets, but that is very often used in passwords'''
nameset.update(['ja']) # 8.10. this would requre changes in main algorithm to put more weight on names and/or frequent words

len(nameset)

32834

In [31]:
'''from a set of substrings, make a list of those which appear as substrings in the target string list'''
def strings_in_target(target, string_set, matchlist):    
    stringlist = []
    for i, tw in enumerate(target): # target word
        slist = []
        for s in string_set:
            ''' 1. string is part of pw, 2. not already in matchlist'''
            if s in tw and s not in matchlist[i]:
                slist.append(s)
        stringlist.append(slist)
    return stringlist

In [32]:
import timeit

'''Include only names that match the passwords'''
start = timeit.default_timer()

name_matchlist = strings_in_target(pwlist,nameset, matchlist)

stop = timeit.default_timer()    
print "Running the function took time ",stop - start," seconds"

Running the function took time  194.910196383  seconds


In [33]:
len(name_matchlist), len(matchlist), len(pwlist)

(38395, 38395, 38395)

### combine matchlist and found names

In [34]:
import itertools
word_name_matchlist = [list(itertools.chain(*x)) for x in zip(matchlist,name_matchlist)]
len(word_name_matchlist)

38395

# Algorithm for the best word combinations

## Functions

Functions:
* def checkCombinations
* def checkDoubles
* def maxN
* def findLongestCombination
* def mylen
* def maybeDoubles
* def chooseSameSize
* def chooseOneCombinationDict (main function)

In [35]:
# All combiword functions together
import operator
import random
import numpy as np
from fuzzywuzzy import fuzz

def checkCombinations(target,target_string, word_list,notfirstone):
    '''checks up to 4 combinations from the word_list 
    which all words together can form the corresponding target word, password'''
    combiwords=[]
    words=word_list
    s=target_string
    for i1,w1 in enumerate(words):
        weight=(len(w1))
        combiwords.append([target, w1,len(w1),(w1)])
        if s.find(w1,s.index(w1)+1)!=-1: #checks doubles
            combiwords.append([target,(w1+w1),(len(w1)+len(w1)),(w1,w1)])
            if s.find(w1+w1,s.index(w1)+1)!=-1: #checks doubles
                combiwords.append([target,(w1+w1+w1),(len(w1)+len(w1)+len(w1)),(w1,w1,w1)])
        for i2,w2 in enumerate(words):
            if s.index(w1)+len(w1)<=s.index(w2) and w1 not in notfirstone:
                weight=(len(w1)+len(w2))
                combiwords.append([target,(w1+w2),(len(w1)+len(w2)),(w1,w2)])
                for i3,w3 in enumerate(words):
                    if (s.index(w1)+len(w1)<=s.index(w2) and s.index(w2)+len(w2)<=s.index(w3)):
                        combiwords.append([target, (w1+w2+w3),(len(w1)+len(w2)+len(w3)),(w1,w2,w3)])
                    for i4,w4 in enumerate(words):
                        if (s.index(w1)+len(w1)<=s.index(w2) and s.index(w2)+len(w2)<=s.index(w3) and 
                            s.index(w3)+len(w3)<=s.index(w4)):
                            weight=(len(w1)+len(w2)+len(w3)+len(w4))
                            combiwords.append([target, (w1+w2+w3+w4),(len(w1)+len(w2)+len(w3)+len(w4)),(w1,w2,w3,w4)])
    return combiwords

def checkDoubles(target_string, word_list,origlist):
    '''checks if the same word appears twice'''
    w1=word_list
    s=target_string
    if s.find(w1,s.index(w1)+1)!=-1: # checks doubles
        return [target_string,(w1+w1),(len(w1)+len(w1)),(origlist+origlist)]
    else:
        return (target_string, w1, len(w1), origlist)

def maxN(datalist):
    '''input list of combiwords and weights
    will sort first from biggest to smallest 
    finds max value(s)
    return max value, or several, if more than max values'''
    if len(datalist)<2: # when only one option exists
        return datalist
    else:
        datalist.sort(key=operator.itemgetter(2), reverse=True)
        myindex=1 # help index
        for i in datalist:
            if datalist[0][2]>datalist[myindex][2]:
                return datalist[:myindex] # return all longest ones
            else:
                myindex=myindex+1
                if (myindex==len(datalist)): # if list contains only the longest ones, stop here
                    return (datalist[:myindex])

def findLongestCombination(target_list, combilist,notfirstone):
    '''Finds the longest combination of words'''    
    all_combis=[]
    for i in xrange(len(combilist)):
        target=target_list[i] 
        starget=target_list[i].lower() # target to lowercase
        if combilist[i]!=[] and len(target_list[i].lower())<100:
            combis=checkCombinations(target, starget, combilist[i], notfirstone)
            all_combis.append(combis)
        else: # no comparisons needed for targets with no matches
            combis=[[target_list[i],"",0,"" ]]
            all_combis.append(combis)
    all_longest=[]
    for i in xrange(len(all_combis)):
        longest=maxN(all_combis[i])
        all_longest.append(longest)
    return all_longest

def mylen(varorlist):
    '''calculates the length of list of matches, 
    if only one word and therefore not list nor tuple
    the length will be 1, which probably is the best match'''
    if type(varorlist) in [tuple,list]:
        mylen=len(varorlist)
    else:
        mylen=1
    #print mylen
    return mylen

def maybeDoubles(n):
    '''catches like namipupunamipupu combo of four words'''
    if n[1] in n[0]:
        k=checkDoubles(n[0],n[1],n[3])
    else:
        k=n
    return k


def chooseSameSize(n,m, target):
    '''checks if combiword has a double like namipupunamipupu and mannepaskamannepaska,
    if same size, choose the one that appears first in the password,
    n= mylist
    m=best '''
    if maybeDoubles(n)[2]>n[2]:
        k = maybeDoubles(n)
    elif maybeDoubles(m)[2]>m[2]:
        k = maybeDoubles(m)
    else:
        if target.find(n[1]) < target.find(m[1]):
            k = n
        else:
            k = m
    return k

def digitsInString(n): 
    return len(filter(lambda x: x.isdigit(), str(n)))

def leftoverChars(target,item):
    newtarget = target.lower() # copy and into 
    if type(item) not in [list,tuple]:
        newtarget = newtarget.replace(item,"")
    else:
        for listitem in item:
            newtarget = newtarget.replace(listitem,"")
    return len(newtarget), digitsInString(newtarget.encode('utf-8')) # 2.10. added .encode('utf-8')

'''Check if password starts with the matched string or first string in the matched list'''
def pwStartsWith(pw,string):
    psw = False
    if type(string) not in [list,tuple]:
        psw = pw.lower().startswith(string)
    else:
        psw = pw.lower().startswith(string[0])
    return psw

'''This is the main function to be called to get a clean dictionary of best hits'''
def chooseOneCombinationDict(target_list, combilist,notfirstone):
    '''against target list, the combination list is checked, using findLongestCombination,
    if more than one longest hit, will choose the shortest list with hits,
    which apparently has longest word(s) and therefore best match
    AND returns list of chosen matches
    this list will be used later for other purpose, but is produced easily here'''
    mylist=findLongestCombination(target_list, combilist,notfirstone)
    onedict={}
    matchlist=[]
    for my in xrange(len(mylist)): # len(mylist)
        templen=99 # temp var for length of the match
        if len(mylist[my])<2: # if only one longest combination
            best=(mylist[my])
        else: # if more than one longest combination
            for ml in mylist[my]:
                if templen>mylen(ml[3]): # first word (length=99), or this one has fewer words (is better)
                    templen=mylen(ml[3])
                    best=ml
                elif templen==mylen(ml[3]): # if size is same, calls choseSameSize
                    best=chooseSameSize(ml, best, mylist[my][0][0])
        if len(best[0][0])==1 and type(best[0]) in [unicode]: # all into same shape, i.e. list of list            
            best=[best]            
        '''password as key,
        best combination, fuzz ration for the best combination'''
        leftover, digits = leftoverChars(best[0][0], best[0][3]) # 7.10. , best[0][3] --> , best[0][1]
        onedict[best[0][0]] = (best[0][1],fuzz.ratio(best[0][1],best[0][0].lower()),\
                               leftover,digits,pwStartsWith(best[0][0],best[0][3]),len(best[0][3]),\
                               int(np.where(min([len(i) for i in best[0][3]])==1, len(best[0][3]),\
                                            min([len(i) for i in best[0][3]]))),\
                               int(np.where(set('[~!@#$%^&*()_+{}":;\']+$-?><=/,.').intersection(best[0][0])>0, \
                                     len(set('[~!@#$%^&*()_+{}":;\']+$-?><=/,.').intersection(best[0][0])), 0)),\
                               best[0][3]) #modified 1.10.16 & 4.10.   
        matchlist.append(best[0][3])
    return onedict, matchlist

## Run the analysis

Uses the list forms, as that was initially build. Did try to modify it to take also dictionary format, but that was nor possible with feasible attempt.

In [36]:
import timeit
start = timeit.default_timer()

notfirstone = ['ja']
oned, onematch = chooseOneCombinationDict(pwlist,word_name_matchlist, notfirstone)

stop = timeit.default_timer()    
print "Running the function took time ",stop - start," seconds"

Running the function took time  14.5815746022  seconds


### Checks

In [37]:
print len(oned)

29587


In [38]:
len(onematch), len(set(onematch))

(38395, 19445)

In [39]:
len(onematch), len(set(onematch))

(38395, 19445)

In [40]:
'''test print of the results'''
oned.items()[:15]

[(u'iittukaunisto', (u'kaunisto', 76, 5, 0, False, 8, 8, 0, u'kaunisto')),
 (u'jussilat', (u'jussila', 93, 1, 0, True, 7, 7, 0, u'jussila')),
 (u'sonja', (u'sonja', 100, 0, 0, True, 5, 5, 0, u'sonja')),
 (u'kultarakas',
  (u'kultarakas', 100, 0, 0, True, 2, 5, 0, (u'kulta', u'rakas'))),
 (u'aenaenNEANEA', (u'enaenne', 74, 5, 0, False, 2, 3, 0, (u'ena', u'enne'))),
 (u'petra1', (u'petra', 91, 1, 1, True, 5, 5, 0, u'petra')),
 (u'protacon', (u'protacon', 100, 0, 0, True, 8, 8, 0, u'protacon')),
 (u'japizkicoolnews',
  (u'coolnews', 70, 7, 0, False, 2, 4, 0, (u'cool', u'news'))),
 (u'aatami001', (u'aatami', 80, 3, 3, True, 6, 6, 0, u'aatami')),
 (u'sitruunamarjut',
  (u'sitruunamarjut', 100, 0, 0, True, 2, 6, 0, (u'sitruuna', u'marjut'))),
 (u'estela12', (u'estela', 86, 2, 2, True, 6, 6, 0, u'estela')),
 (u'yojikkoira', (u'koira', 67, 5, 0, False, 5, 5, 0, u'koira')),
 (u'camillaviivi',
  (u'camillaviivi', 100, 0, 0, True, 2, 5, 0, (u'camilla', u'viivi'))),
 (u'nalle2', (u'nalle', 91, 1, 

In [41]:
'''test print of the same values, 
original list of all matches per key values, i.e. pw
shows how well the algorithm works, or does not work'''
for k in oned.keys()[:15]:
    print d0[k.encode('utf-8')] # scandics require the .encode('utf-8')

[u'uni', u'kaunis', u'kaunisto']
[u'jussi', u'jussila', u'silat']
[u'sonja']
[u'kulta', u'tara', u'rakas']
[u'enne']
[u'petra', u'petr']
[u'protacon', u'ota', u'pro', u'prota']
[u'new', u'cool', u'news']
[u'tami', u'aatami']
[u'ruuna', u'marjut', u'sitruuna', u'marju', u'sit']
[u'tela', u'este']
[u'koira']
[u'mill', u'camilla', u'milla', u'viivi']
[u'nalle', u'all', u'alle']
[u'aave', u'taavetti']


In [42]:
oned.values()[15][0], oned.values()[15][1], oned.values()[15][2],oned.values()[15][3]

(u'herneherkku', 92, 2, 2)

### Before storing result in file, replace @ with AT and , with DOT to ignore transformation problems

In [43]:
import pandas as pd
import numpy as np

In [44]:
df_oned = pd.DataFrame({'pw' : oned.keys() , 'values' : oned.values() })

In [45]:
df_oned.head(5)

,pw,values
0,iittukaunisto,"(kaunisto, 76, 5, 0, False, 8, 8, 0, kaunisto)"
1,jussilat,"(jussila, 93, 1, 0, True, 7, 7, 0, jussila)"
2,sonja,"(sonja, 100, 0, 0, True, 5, 5, 0, sonja)"
3,kultarakas,"(kultarakas, 100, 0, 0, True, 2, 5, 0, (kulta,..."
4,aenaenNEANEA,"(enaenne, 74, 5, 0, False, 2, 3, 0, (ena, enne))"


In [46]:
df_oned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29587 entries, 0 to 29586
Data columns (total 2 columns):
pw        29587 non-null object
values    29587 non-null object
dtypes: object(2)
memory usage: 462.4+ KB


In [47]:
df_oned['combiword'] = [a for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['fuzzy ratio'] = [b for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['#leftover chars'] = [c for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['#leftover numbers'] = [d for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['leftovers are numbers'] = [c==d for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['pw starts with match'] = [e for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['matched words'] = [i for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['min word length'] = [g for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['list'] = np.where(df_oned['matched words'].str.contains(","), True, False)
length = [f for a, b, c, d, e, f, g, h, i in df_oned['values']]
df_oned['#matches'] = np.where(df_oned['list'], length, 1)
df_oned['#special chars'] = [h for a, b, c, d, e, f, g, h, i in df_oned['values']]

In [48]:
df_oned.head(10)

,pw,values,combiword,fuzzy ratio,#leftover chars,#leftover numbers,leftovers are numbers,pw starts with match,matched words,min word length,list,#matches,#special chars
0,iittukaunisto,"(kaunisto, 76, 5, 0, False, 8, 8, 0, kaunisto)",kaunisto,76,5,0,False,False,kaunisto,8,False,1,0
1,jussilat,"(jussila, 93, 1, 0, True, 7, 7, 0, jussila)",jussila,93,1,0,False,True,jussila,7,False,1,0
2,sonja,"(sonja, 100, 0, 0, True, 5, 5, 0, sonja)",sonja,100,0,0,True,True,sonja,5,False,1,0
3,kultarakas,"(kultarakas, 100, 0, 0, True, 2, 5, 0, (kulta,...",kultarakas,100,0,0,True,True,"(kulta, rakas)",5,True,2,0
4,aenaenNEANEA,"(enaenne, 74, 5, 0, False, 2, 3, 0, (ena, enne))",enaenne,74,5,0,False,False,"(ena, enne)",3,True,2,0
5,petra1,"(petra, 91, 1, 1, True, 5, 5, 0, petra)",petra,91,1,1,True,True,petra,5,False,1,0
6,protacon,"(protacon, 100, 0, 0, True, 8, 8, 0, protacon)",protacon,100,0,0,True,True,protacon,8,False,1,0
7,japizkicoolnews,"(coolnews, 70, 7, 0, False, 2, 4, 0, (cool, ne...",coolnews,70,7,0,False,False,"(cool, news)",4,True,2,0
8,aatami001,"(aatami, 80, 3, 3, True, 6, 6, 0, aatami)",aatami,80,3,3,True,True,aatami,6,False,1,0
9,sitruunamarjut,"(sitruunamarjut, 100, 0, 0, True, 2, 6, 0, (si...",sitruunamarjut,100,0,0,True,True,"(sitruuna, marjut)",6,True,2,0


## Save best matches as a file

In [49]:
'''temporary file for csv saving
In order to make csv readable, modify special marks:
','-->'.' 
'\''-->'.' 
'\"'-->'.' 
';'-->'.'
'''

df_oned_savecsv = df_oned.copy(deep=True)

In [50]:
df_oned_savecsv['pw'] = np.where(df_oned_savecsv['pw'].str.contains(","), df_oned_savecsv['pw'].str.replace(",", "."), df_oned_savecsv['pw'])
df_oned_savecsv['pw'] = np.where(df_oned_savecsv['pw'].str.contains("\'"), df_oned_savecsv['pw'].str.replace("\'", "."), df_oned_savecsv['pw'])
df_oned_savecsv['pw'] = np.where(df_oned_savecsv['pw'].str.contains("\""), df_oned_savecsv['pw'].str.replace("\"", "."), df_oned_savecsv['pw'])
df_oned_savecsv['pw'] = np.where(df_oned_savecsv['pw'].str.contains(";"), df_oned_savecsv['pw'].str.replace(";", "."), df_oned_savecsv['pw'])

In [51]:
df_oned_savecsv.head(2)

,pw,values,combiword,fuzzy ratio,#leftover chars,#leftover numbers,leftovers are numbers,pw starts with match,matched words,min word length,list,#matches,#special chars
0,iittukaunisto,"(kaunisto, 76, 5, 0, False, 8, 8, 0, kaunisto)",kaunisto,76,5,0,False,False,kaunisto,8,False,1,0
1,jussilat,"(jussila, 93, 1, 0, True, 7, 7, 0, jussila)",jussila,93,1,0,False,True,jussila,7,False,1,0


In [52]:
df_oned_savecsv = df_oned_savecsv.drop(['values'], axis=1); # without semicolon the dataset would be printed below

In [53]:
df_oned_savecsv.head(2)

,pw,combiword,fuzzy ratio,#leftover chars,#leftover numbers,leftovers are numbers,pw starts with match,matched words,min word length,list,#matches,#special chars
0,iittukaunisto,kaunisto,76,5,0,False,False,kaunisto,8,False,1,0
1,jussilat,jussila,93,1,0,False,True,jussila,7,False,1,0


In [54]:
filename = "tko_2027/data/"+sourcename+"_before_validation_pwsandmatches.csv"
df_oned_savecsv.to_csv(filename, sep=',', encoding='utf-8', index=False)

In [55]:
del df_oned_savecsv

In [56]:
'''Remove extra column'''
df_oned = df_oned.drop(['values'], axis=1); # without semicolon the dataset would be printed below

## Filter/select correct matches
NOTE! These need to be in this exact order!

In [57]:
df_temp = df_oned.copy(deep=True)

In [58]:
len(df_temp), len(df_oned)

(29587, 29587)

In [59]:
df_valid = []

In [60]:
'''All valid matches:
* Leftovers are numbers TRUE'''
df_valid = df_temp[df_temp['leftovers are numbers']]
len(df_valid)

17286

In [61]:
df_temp = df_temp[~(df_temp['leftovers are numbers'])]
len(df_temp), len(df_oned)-len(df_valid)

(12301, 12301)

In [62]:
# df_new = pd.concat([df_a, df_b])
# df_new

In [63]:
'''All valid:
* leftovers are numbers FALSE
* PW start FALSE
* leftover chars == 2
* first chars are in spec_list'''

spec_list = ['is','in','en']

mytemp = df_temp[(~(df_temp['leftovers are numbers'])) & (~(df_temp['pw starts with match'])) & \
        (df_temp['#leftover chars']==2) & (df_temp['pw'].str[0:2].isin(spec_list))]

In [64]:
df_valid = pd.concat([df_valid, mytemp])
len(df_valid)

17298

In [65]:
'''remove same data as complement from df_temp'''
df_temp = df_temp[~((~(df_temp['leftovers are numbers'])) & (~(df_temp['pw starts with match'])) & \
        (df_temp['#leftover chars']==2) & (df_temp['pw'].str[0:2].isin(spec_list)))]
len(df_temp), len(df_oned)-len(df_valid)

(12289, 12289)

In [66]:
'''All valid:
* fuzzy ratio == 100
* min word length >=4'''

mytemp = df_temp[(df_temp['fuzzy ratio']==100) & (df_temp['min word length']>=4)]

In [67]:
df_valid = pd.concat([df_valid, mytemp])
len(df_valid)

17309

In [68]:
'''remove same data as complement from df_temp'''
df_temp = df_temp[~((df_temp['fuzzy ratio']==100) & (df_temp['min word length']>=4))]
len(df_temp), len(df_oned)-len(df_valid)

(12278, 12278)

In [69]:
'''NOT valid:
* PW start TRUE
* min word length ==3'''

df_drop = df_temp[(df_temp['pw starts with match']) & (df_temp['min word length']==3)]

In [70]:
len(df_drop)

1520

In [71]:
'''remove same data as complement from df_temp'''
df_temp = df_temp[~((df_temp['pw starts with match']) & (df_temp['min word length']==3))]
len(df_temp), len(df_oned)-len(df_valid)-len(df_drop)  

(10758, 10758)

In [72]:
'''All valid:
* PW start TRUE
* min word length > 3'''

mytemp = df_temp[(df_temp['pw starts with match']) & (df_temp['min word length']>3)]
len(mytemp)

4530

In [73]:
df_valid = pd.concat([df_valid, mytemp])
len(df_valid)

21839

In [74]:
'''remove same data as complement from df_temp'''
df_temp = df_temp[~((df_temp['pw starts with match']) & (df_temp['min word length']>3))]
len(df_temp), len(df_oned)-len(df_valid)

(6228, 7748)

In [75]:
'''All valid:
* fuzzy ratio >=85
* list FALSE'''

mytemp = df_temp[(~(df_temp['list'])) & (df_temp['fuzzy ratio']>=85)]
len(mytemp)

402

In [76]:
df_valid = pd.concat([df_valid, mytemp])
len(df_valid)

22241

In [77]:
'''remove same data as complement from df_temp'''
df_temp = df_temp[(~(~(df_temp['list'])) & (df_temp['fuzzy ratio']>=85))]
len(df_temp), len(df_oned)-len(df_valid)

(1132, 7346)

### Store valid matches

In [78]:
# filename = "tko_2027/data/"+sourcename+"_valid_file_content.csv"
# df_valid.to_csv(filename, sep=',', encoding='utf-8', index=False)

## Get all the valid words

In [79]:
'''Itearative function for flattening the list of strings and lists as one list
* if item is a tuple; call function iteratively
* else item is string; add to new list'''
def flatten(mylist):
    newlist = []
    for item in mylist:
        if isinstance(item, tuple) or isinstance(item, list):
            newlist.extend(flatten(item))            
        else:
            newlist.append(item)
    return newlist

In [80]:
selected_pws = flatten(df_valid['pw'])
selected_words = flatten(df_valid['matched words'])
len(selected_pws),len(selected_words)

(22241, 29553)

In [81]:
wordsum_dict={}
for selword in set(selected_words):
    wordsum_dict[selword]=0
    for i in selected_pws:
        if (type(oned[i][-1])==tuple and selword in oned[i][-1]) or (type(oned[i][-1])==unicode and selword == oned[i][-1]):
            wordsum_dict[selword] = wordsum_dict[selword]+allpw_counter[i]


In [82]:
len(wordsum_dict), len(selected_words), len(set(selected_words))

(11749, 29553, 11749)

In [83]:
#from collections import Counter

'''count all occurrences of unique words in the selected words (to be used later)
print out 10 most common ones'''
word_pwfreq = Counter(selected_words)
pw_counter = Counter(pwlist)
#allpw_counter = Counter(allpws) # done earlier already
Counter(selected_words).most_common(10)

[(u'koira', 72),
 (u'man', 70),
 (u'kakka', 67),
 (u'paska', 65),
 (u'love', 57),
 (u'kissa', 56),
 (u'ville', 55),
 (u'sami', 52),
 (u'make', 51),
 (u'mikko', 51)]

In [84]:
len(word_pwfreq), len(set(selected_words)), len(pwlist), len(pw_counter), len(allpws), len(allpw_counter)

(11749, 11749, 38395, 29587, 38395, 29587)

In [85]:
'''the most common ones before validation'''
Counter(flatten(matchlist)).most_common(10)

[(u'ari', 1024),
 (u'ala', 884),
 (u'eri', 790),
 (u'all', 785),
 (u'per', 710),
 (u'san', 706),
 (u'one', 575),
 (u'min', 508),
 (u'las', 495),
 (u'sam', 431)]

### XXX

# Identify the Finnish words

## Query the needed words from morphological analysis

File filtered_vocab_omor.gz, located on a Linux server, contains morphological analysis. This ready made analysis file contains Finnish language morphological analysis, so this file can be utilized to identify Finnish words in the password matchlists.
Below a short desciption of the query tools used to get the needed information and file from utu Linux server.

Getting uniq Finnish words from filtered_vocab_omor.gz and storing the result into filtered_vocab_omor_uniq.gz

Example lines from filtered_vocab_omor.gz (inluces empty lines and non-Finnish words that end with +?):
<blockquote>
        yhteisöllisin   yhteisö<N><Der_llinen><A><Pl><Ins><cap> 0.0<br>
        yhteisöllisin   yhteisöllinen<A><Pos><Pl><Ins><cap>     0.0<br>
        yhteisöllisin   yhteisöllinen<A><Superl><Sg><Nom><cap>  0.0<<br>
        viewpoint       +?<br>
        supercrossiin   +?<br>
        tovoitteet      +?<br>
</blockquote>     

In order to get unique words, need to
* grep lines that start with words, as the file contains also empty lines
* get only the first column
* sort
* get unique words and their count
* sort in reverse order (biggest frequency first)
* for visual inspection, only 10 first are needed

~~~~~~~~~~~~~~~~~~~~
$ zcat filtered_vocab_omor.gz | grep -P '^[a-zåäöA-ZÅÄÖ]' | cut -f 1 | sort | uniq -c | sort -nr | head<br></code>
~~~~~~~~~~~~~~~~~~~~
<blockquote>
        1152 isoisoisoisoisoisoisovanhempani
        512 häähäähäähäähäähäähäähäähää
        432 isoisoisoisoisoisoäitini
        384 iso-iso-iso-iso-iso-iso-iso-vanhempien
        216 isoisovanhempansa
        216 isoisoisoisoisovanhemmat
        216 isoisoisoisänsä
        216 isoisoisoäitinsä
        192 toimittaja-kirjailija-lääketiedekirjoittaja-kääntäjä-copywriter-valokuvaaja
        180 isovanhempansa
</blockquote>

This one however included also non-Finnish words.

Filtering first columns of lines that include '0.0', as all those those lines start with Finnish words:

~~~~~~~~~~~~~~~~~~~~
$ zcat filtered_vocab_omor.gz | grep -P '0.0' | cut -f 1 | sort | uniq | head
~~~~~~~~~~~~~~~~~~~~
<blockquote>        
        a-aalto
        a-äänellä
        a-äänestysalueen
        a-äänet
        a-ääni
        a-äänistä
        a-äänne
        ä-äänne
        a-äännettä
        a-äänteeksi
</blockquote>

Storing the query to a zip file for further usage:

~~~~~~~~~~~~~~~~~~~~
zcat filtered_vocab_omor.gz | grep -P '0.0' | cut -f 1 | sort | uniq | gzip > filtered_vocab_omor_uniq.gz
~~~~~~~~~~~~~~~~~~~~

Just out of curiosity, how many lines in the original file?

~~~~~~~~~~~~~~~~~~~~
$ zcat filtered_vocab_omor.gz | wc -l
~~~~~~~~~~~~~~~~~~~~
<blockquote>
    31962465
</blockquote>

How many non-empty lines in the original file?

~~~~~~~~~~~~~~~~~~~~
$ zcat filtered_vocab_omor.gz | grep -v "^[[:space:]]*$" | wc -l
~~~~~~~~~~~~~~~~~~~~
<blockquote>
    20227331
</blockquote>

Number of lines in the new file (one word per line, nothing else):

~~~~~~~~~~~~~~~~~~~~
$ zcat filtered_vocab_omor_uniq.gz | wc -l
~~~~~~~~~~~~~~~~~~~~
<blockquote>
    6924238 
</blockquote>
So out of 20 million words the number of unique Finnish words is 6,9 million.

Copying from linux server to win computer:

~~~~~~~~~~~~~~~~~~~~
pscp -l elmasyr TY1******.utu.fi:/home/elmasyr/password_analyzer/filtered_vocab_omor.gz C:\mytemp
~~~~~~~~~~~~~~~~~~~~

Need to now check the found words vs. filtered_vocab_omor_uniq.gz

## Reading omor_uniq

In [86]:
import timeit

In [87]:
def readgz_singlecol_tolist(filename):
    '''reads gz file that has words in first column,
    no other columns in the file,
    returns file content as list in utf-8 format'''
    fwrds=[]
    with gzip.open(filename, 'rb') as f:
        for line in f:
            word =line.split()
            fwrds.append(word) #word could be replaced with i.decode('utf-8') for i in word, 
                                #but seemmed to take long time, so below the fix

    f.close()
    #wordlist to unicode
    finnwords=[x[0].decode('utf-8') for x in fwrds]
    #print len(fwrds), len(finnwords) # check
    return finnwords

In [88]:
start = timeit.default_timer()

finnwords = readgz_singlecol_tolist('filtered_vocab_omor_uniq.gz')

stop = timeit.default_timer()    
print "Running the function took time ",stop - start," seconds"

Running the function took time  52.1509844257  seconds


In [89]:
print [i for i in finnwords[:5]]

[u'a-aalto', u'a-\xe4\xe4nell\xe4', u'a-\xe4\xe4nestysalueen', u'a-\xe4\xe4net', u'a-\xe4\xe4ni']


In [90]:
for i in finnwords[:10]:    
    print i
    #i[0].encode("utf-8") #decode needed in order to print to file

a-aalto
a-äänellä
a-äänestysalueen
a-äänet
a-ääni
a-äänistä
a-äänne
ä-äänne
a-äännettä
a-äänteeksi


In [91]:
print finnwords[:5]

[u'a-aalto', u'a-\xe4\xe4nell\xe4', u'a-\xe4\xe4nestysalueen', u'a-\xe4\xe4net', u'a-\xe4\xe4ni']


##  Matches vs. omor_uniq

Note! the list of unique passwords in this testing round (using matches_round1.gz) is 43870, but the number of unique matches is only 15148 (!).

When looking this further, notices that most of the matches are empty lists, i.e. [] appears 16650 times (xxx need to check this as the value deviates from the check part below), word "akka" appears 390 times, "you" 68 times etc.

## The function and call:

In [92]:
def finnwords_tolist(list_x,list_y):
    '''get dictionary (with pw as keys, words as values)
    and list of words,
    return intersection of the values and words as list'''
    s1=set(list_x)
    s2=set(finnwords)
    s3 = s1 & s2
    return list(s3)

In [93]:
finnwlist = finnwords_tolist(selected_words, finnwords)
len(finnwlist)

4937

# Getting the frequencies

<code>
$ zcat all_vocab.gz | wc -l
28399265
</code>

Importing all_vocab to dictionary format

For the following function, need too decide if all macthes need to be stored.
Currently stores the last occurrence.

<blockquote>
        $ zcat all_vocab | grep -P '^yellow\t'
        yellow  PROPN   _       1233
        yellow  NOUN    Case=Par|Number=Sing    2
        yellow  NOUN    Case=Nom|Number=Sing    138
        yellow  ADV     _       40
        yellow  SYM     _       27
        yellow  X       Foreign=Foreign 350
        yellow  ADJ     Case=Nom|Degree=Pos|Number=Sing 14
        yellow  PROPN   Case=Nom|Number=Sing    862
        yellow  NOUN    _       2
        yellow  NUM     Case=Nom|Number=Sing|NumType=Card       2
        yellow  INTJ    _       4
</blockquote>

## Reading the frequencies

### Get frequency and related morphological analysis
Note! Stores only the last occurrence of the word and related values

In [94]:
# def get_frequencies(filename, lookup_set):
#     '''get web crawled data in gz with tab delimiter (word, pos, analysis, freq)
#     all the first column words that match the lookup set content
#     will be stored in a dictionary, including frequency and other values
#     '''
#     dw={}
#     n=0 # will be removed later
#     with gzip.open(filename, 'rb') as f:    
#         for line in f:        
#             line=line.rstrip("\n") # removes the line end mark
#             word, pos, analysis, freq=line.split("\t")        
#             word=word.decode('utf-8') # convert word to unicode
#             '''additional check that the function proceeds
#             will remove this and all related in later versions''' 
#             n+=1
#             if n%1000000 == 0:
#                 ntimer=timeit.default_timer()
#                 print n, ntimer-start
#             if word in lookup_set: # using set s1 here, as set is much faster than list (matched)
#                 dw[word]=(int(freq), pos, analysis)# pos, analysis
#                 '''note! currently stores the last occurrence of the 
#                 word and related values
#                 -- needs tuning'''       
#     f.close()
#     return dw

In [95]:
# # zcat all_vocab.gz | wc -l
# # 28399265 - the amount of lines in the file

# start = timeit.default_timer()

# dw = get_frequencies('all_vocab.gz', set(selected_words))

# stop = timeit.default_timer()    
# print "Running the function took time ",stop - start," seconds"

### Get sum of frequencies per word
Only stores the frequency of the word.
If the word has several different morhological results, a cumulative frequency is calculated.

In [96]:
def get_freq_sums(filename, lookup_set):
    '''get web crawled data in gz with tab delimiter (word, pos, analysis, freq)
    all the first column words that match the lookup set content
    will be stored in a dictionary, including frequency and other values
    '''
    dw={}
    n=0 # will be removed later
    with gzip.open(filename, 'rb') as f:    
        for line in f:        
            line=line.rstrip("\n") # removes the line end mark
            word, freq=line.split("\t")        
            word=word.decode('utf-8') # convert word to unicode
            freqn=int(freq)
            '''additional check that the function proceeds
            will remove this and all related in later versions''' 
            n+=1
            if n%1000000 == 0:
                ntimer=timeit.default_timer()
                print n, ntimer-start
            '''using set s1 here, as set is much faster than list (matched)'''
            if word in lookup_set: 
                if word in dw.keys():
                    dw[word]=dw[word]+freqn
                else:
                    dw[word]=freqn
                '''stores the sum of frequencies'''       
    f.close()
    return dw

In [97]:
# zcat all_vocab.gz | wc -l
# 28399265 - the amount of lines in the file

start = timeit.default_timer()

dw_freqsum = get_freq_sums('all_vocab_freq.gz', set(selected_words))

stop = timeit.default_timer()    
print "Running the function took time ",stop - start," seconds"

1000000 6.67347181963
2000000 13.3870037106
3000000 20.2917551354
4000000 27.3386040152
5000000 34.5794901238
6000000 41.8180762363
7000000 49.107537077
8000000 56.4584199015
9000000 63.9484268346
10000000 71.3478912331
11000000 79.1019963328
12000000 86.7148666185
13000000 94.1976451552
14000000 101.691108354
15000000 109.284726863
16000000 116.891177799
17000000 124.520095603
18000000 132.174340162
19000000 139.853485002
20000000 147.502804581
21000000 155.035435562
22000000 162.654502838
23000000 170.561364398
24000000 178.841007779
25000000 187.345284495
26000000 195.274665377
27000000 203.171802412
28000000 211.301358218
Running the function took time  214.353535745  seconds


### Get morphological analysis
Morphological analysis is chosen by the biggest frequncy of the used analysis data; assumed that the most used wrod and it's analysis would be most suitable for the word.
In most cases this seems to go right, but due to the nature of words used in passwords e.g. "anna" is usually a proper noun (name) in passwords, but in web data more ofter verb.

In [98]:
def get_omorph_analysis(filename, lookup_set):
    '''get web crawled data in gz with tab delimiter (word, pos, analysis, freq)
    all the first column words that match the lookup set content
    will be stored in a dictionary, including frequency and other values
    '''
    dw={}
    n=0 # will be removed later
    with gzip.open(filename, 'rb') as f:    
        for line in f:        
            line=line.rstrip("\n") # removes the line end mark
            word, pos, analysis, freq=line.split("\t")               
            word=word.decode('utf-8') # convert word to unicode
            freqn=int(freq)
            '''additional check that the function proceeds
            will remove this and all related in later versions''' 
            n+=1
            if n%1000000 == 0:
                ntimer=timeit.default_timer()
                print n, ntimer-start
            '''using set here, as set is much faster than list (matched)'''
            if word in lookup_set: 
                if word in dw.keys():                    
#                     print word, dw[word][0]
#                     print freq
                    if dw[word][0]>int(freq):
#                        print 'vanha parempi'
                        continue
                    else:
#                        print 'uusi arvo', freq
                        dw[word]=(int(freq), pos, analysis)
                else:
#                    print word
                    dw[word]=(int(freq), pos, analysis)
                '''stores values for the bigger frequency'''       
    f.close()
    return dw


In [99]:
# zcat all_vocab.gz | wc -l
# 28399265 - the amount of lines in the file

start = timeit.default_timer()

dw_omor = get_omorph_analysis('all_vocab.gz', set(selected_words))

stop = timeit.default_timer()    
print "Running the function took time ",stop - start," seconds"

1000000 7.1340394865
2000000 14.4941582151
3000000 22.8838650198
4000000 30.7591492409
5000000 40.1492808093
6000000 49.4504280707
7000000 58.3873201453
8000000 66.4626236376
9000000 74.5755378251
10000000 83.5707565243
11000000 91.9726189852
12000000 100.130294723
13000000 110.025101544
14000000 118.442906046
15000000 127.031091569
16000000 135.645868445
17000000 144.208971809
18000000 153.00332228
19000000 161.279544744
20000000 170.747271249
21000000 179.136584648
22000000 187.206251041
23000000 195.318195968
24000000 203.457190652
25000000 211.577474634
26000000 219.743871428
27000000 227.89405821
28000000 235.887087523
Running the function took time  239.136960949  seconds


### Checks

In [100]:
print len(dw_freqsum)#, dw_freqsum['yellow']

11742


In [101]:
dw_freqsum.items()[:10]

[(u'toisi', 16427),
 (u'toise', 198),
 (u'yellow', 2674),
 (u'factory', 3074),
 (u'four', 5076),
 (u'woods', 4603),
 (u'jihaa', 272),
 (u'mirage', 556),
 (u'woody', 2090),
 (u'suzana', 18)]

In [102]:
print len(dw_omor)

11742


In [103]:
dw_omor.items()[:10]

[(u'toisi',
  (15822, 'VERB', 'Mood=Cnd|Number=Sing|Person=3|VerbForm=Fin|Voice=Act')),
 (u'toise', (114, 'ADV', '_')),
 (u'yellow', (1233, 'PROPN', '_')),
 (u'factory', (1954, 'PROPN', 'Case=Nom|Number=Sing')),
 (u'four', (2442, 'PROPN', '_')),
 (u'woods', (4020, 'PROPN', 'Case=Nom|Number=Sing')),
 (u'jihaa', (221, 'NOUN', 'Case=Par|Number=Sing')),
 (u'mirage', (396, 'PROPN', 'Case=Nom|Number=Sing')),
 (u'woody', (1534, 'PROPN', '_')),
 (u'suzana', (8, 'PROPN', '_'))]

# Combine matched words related information in one list

In [104]:
len(selected_words)

29553

In [105]:
'''Related to the names, 
a) checking that the types are identical
b) Just out of curiosity, how many names in the matchset?
'''
print type(namelist[0]), type(selected_words[0])
print len([item for item in selected_words if item in namelist])

<type 'unicode'> <type 'unicode'>
13782


## match_freq_finn_names

Features:
* The found match
* Frequency the match has been used in these passwords
* Sum of frequencies, match has been used in web text
* Is this a Finnish word?
* Is this a Finnish name?

In [106]:
# Names do not necessarily have morphological analysis, so those are separated
wordset = set(selected_words)
wordset1 = [item for item in wordset if item in dw_omor.keys()]
wordset2 = [item for item in wordset if item not in dw_omor.keys()]
len(wordset),len(wordset1),len(wordset2), len(selected_words)

(11749, 11742, 7, 29553)

In [107]:
# Combining all needed details for storing the data

start = timeit.default_timer()

match_freq_finn_names2=[(item, wordsum_dict[item], 0, \
                        0, 'PROPN', 'Case=Nom|Number=Sing',\
                        item in finnwords, item in namelist) for item in wordset2]


match_freq_finn_names1=[(item, wordsum_dict[item], dw_freqsum[item], \
                        dw_omor[item][0], dw_omor[item][1], dw_omor[item][2],\
                        item in finnwords, item in namelist) for item in wordset1]

match_freq_finn_names = match_freq_finn_names1 + match_freq_finn_names2

print len(match_freq_finn_names), len(match_freq_finn_names1), len(match_freq_finn_names2)

stop=timeit.default_timer()
print stop-start

11749 11742 7
2045.41043756


In [108]:
match_freq_finn_names[:10]

[(u'toisi',
  2,
  16427,
  15822,
  'VERB',
  'Mood=Cnd|Number=Sing|Person=3|VerbForm=Fin|Voice=Act',
  True,
  False),
 (u'toise', 1, 198, 114, 'ADV', '_', False, False),
 (u'sonja', 16, 8760, 8530, 'PROPN', 'Case=Nom|Number=Sing', False, True),
 (u'factory', 1, 3074, 1954, 'PROPN', 'Case=Nom|Number=Sing', False, False),
 (u'four', 3, 5076, 2442, 'PROPN', '_', False, False),
 (u'woods', 1, 4603, 4020, 'PROPN', 'Case=Nom|Number=Sing', False, True),
 (u'jihaa', 3, 272, 221, 'NOUN', 'Case=Par|Number=Sing', False, False),
 (u'mirage', 3, 556, 396, 'PROPN', 'Case=Nom|Number=Sing', False, False),
 (u'woody', 2, 2090, 1534, 'PROPN', '_', False, False),
 (u'suzana', 1, 18, 8, 'PROPN', '_', False, True)]

DW

[(u'myy', 4, 50973, (2, 'SCONJ', '_'), True, True),
 (u'luuranko', 1, 1867, (26, 'PROPN', 'Case=Nom|Number=Sing'), True, False),
 (u'devils', 1, 1678, (1120, 'PROPN', 'Case=Nom|Number=Sing'), False, False),
 (u'rinkula', 1, 415, (6, 'PROPN', 'Case=Nom|Number=Sing'), True, False),
 (u'sonja', 2, 8760, (2, 'NOUN', 'Case=Par|Number=Plur'), False, True),
 (u'topikissa', 1, 373, (354, 'NOUN', 'Case=Ine|Number=Sing'), False, False),
 (u'pikku',9, 92129, (231, 'ADJ', 'Case=Nom|Degree=Pos|Number=Sing'),True, False),
 (u'unelma', 2, 24240, (1411, 'PROPN', 'Case=Nom|Number=Sing'), True, True),
 (u'iida', 1, 9383, (90, 'NOUN', 'Case=Nom|Number=Sing'), False, True),
 (u'pulse', 1, 714, (75, 'NOUN', 'Case=Nom|Number=Sing'), False, False)]
 
DW_omor

[(u'myy',4, 50973, (39400,'VERB','Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act'),True,True),
 (u'luuranko', 1, 1867, (1833, 'NOUN', 'Case=Nom|Number=Sing'), True, False),
 (u'devils', 1, 1678, (1120, 'PROPN', 'Case=Nom|Number=Sing'), False, False),
 (u'rinkula', 1, 415, (409, 'NOUN', 'Case=Nom|Number=Sing'), True, False),
 (u'sonja', 2, 8760, (8530, 'PROPN', 'Case=Nom|Number=Sing'), False, True),
 (u'topikissa', 1, 373, (354, 'NOUN', 'Case=Ine|Number=Sing'), False, False),
 (u'pikku', 9, 92129, (80639, 'ADV', '_'), True, False),
 (u'unelma', 2, 24240, (22829, 'NOUN', 'Case=Nom|Number=Sing'), True, True),
 (u'iida', 1, 9383, (9248, 'PROPN', 'Case=Nom|Number=Sing'), False, True),
 (u'pulse', 1, 714, (367, 'PROPN', 'Case=Nom|Number=Sing'), False, False)]

## Save match_freq_finn_names as a file

In [109]:
filename = "tko_2027/data/"+sourcename+"_matchfreqfinnw_names.csv"
print filename

tko_2027/data/78k_matchfreqfinnw_names.csv


In [110]:
dfmatch_names=pd.DataFrame(match_freq_finn_names, columns = ["matched word", "freq in pws", "freq in web",\
                                                             "individual freq", "POS", "analysis", "finnish word", "finnish name"])
filename = "tko_2027/data/"+sourcename+"_matchfreqfinnw_names.csv"
dfmatch_names.to_csv(filename, sep=',', encoding='utf-8', index=False)

In [111]:
pwset = set(allpws)
pwset1 = [item for item in pwset if item not in selected_pws]
pwset2 = [item for item in pwset if item in selected_pws]
len(allpws),len(pwset),len(pwset1),len(pwset2), len(selected_pws), len(pwset1)+len(pwset2)

(38395, 29587, 7346, 22241, 22241, 29587)

In [112]:
pwset1[0], allpw_counter[pwset1[0]]

(u'iittukaunisto', 1)

In [113]:
# just to compare set and list iteration complexity
start = timeit.default_timer()

pwfinal1=[(item, pw_counter[item], '', '',sourcename) for item in pwset1] # allpw_counter --> pw_counter

pwfinal2=[(item, pw_counter[item], oned[item][0], oned[item][-1],sourcename) for item in pwset2]

pwfinal = pwfinal1 + pwfinal2

print len(pwfinal), len(pwfinal1), len(pwfinal2)

stop=timeit.default_timer()
print stop-start

29587 7346 22241
0.286656259479


In [114]:
len(pwfinal1), len(pwfinal2)#, len(pwfinal3)

(7346, 22241)

In [115]:
pwfinal1[1],pwfinal2[1], pwfinal[1]

((u'aenaenNEANEA', 1, '', '', '78k'),
 (u'sonja', 7, u'sonja', u'sonja', '78k'),
 (u'aenaenNEANEA', 1, '', '', '78k'))

In [116]:
dfmatch_names=pd.DataFrame(pwfinal, columns = ["pw", "freq in pws","combiword","found words","source"])

dfmatch_names['pw'] = np.where(dfmatch_names['pw'].str.contains(","), dfmatch_names['pw'].str.replace(",", "."), dfmatch_names['pw'])
dfmatch_names['pw'] = np.where(dfmatch_names['pw'].str.contains("\""), dfmatch_names['pw'].str.replace("\"", "."), dfmatch_names['pw'])
dfmatch_names['pw'] = np.where(dfmatch_names['pw'].str.contains("\'"), dfmatch_names['pw'].str.replace("\'", "."), dfmatch_names['pw'])
dfmatch_names['pw'] = np.where(dfmatch_names['pw'].str.contains(";"), dfmatch_names['pw'].str.replace(";", "."), dfmatch_names['pw'])

# len(dfmatch_names)
# dfmatch_names = dfmatch_names[dfmatch_names['freq in pws']]
# len(dfmatch_names)

filename = "tko_2027/data/"+sourcename+"_matchfreqfinnw_allpw.csv"
dfmatch_names.to_csv(filename, sep=',', encoding='utf-8', index=False)

In [117]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

In [118]:
import io, json

filename = "tko_2027/data/"+sourcename+"_word_counter.txt"
with io.open(filename, 'w', encoding='utf-8') as f:
  f.write(unicode(json.dumps(wordsum_dict, ensure_ascii=False)))

In [119]:
import io, json

# filename = "tko_2027/data/"+sourcename+"_word_counter.txt"
# with io.open(filename, 'w', encoding='utf-8') as f:
#   f.write(unicode(json.dumps(word_pwfreq, ensure_ascii=False)))

filename = "tko_2027/data/"+sourcename+"_allpw_counter.txt"
with io.open(filename, 'w', encoding='utf-8') as f:
  f.write(unicode(json.dumps(allpw_counter, ensure_ascii=False)))

filename = "tko_2027/data/"+sourcename+"_pw_counter.txt"
with io.open(filename, 'w', encoding='utf-8') as f:
  f.write(unicode(json.dumps(pw_counter, ensure_ascii=False)))